In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from markdownify import markdownify as md
import os

class ShatteredPixelSpider(scrapy.Spider):
    name = 'shattered_pixel'
    allowed_domains = ['pixeldungeon.fandom.com']
    start_urls = ['https://pixeldungeon.fandom.com/wiki/Shattered_Pixel_Dungeon']

    # Criando a pasta 'scrap' se não existir
    if not os.path.exists('scrap'):
        os.makedirs('scrap')

    def parse(self, response):
        # Extraindo o texto da página e convertendo para Markdown
        content = response.css('#mw-content-text').get()
        if content:
            markdown_text = md(content)
            page = response.url.split("/")[-1]
            filename = os.path.join('scrap', f'{page}.md')
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(markdown_text)
            self.log(f'Saved file {filename}')

        # Seguindo apenas links que estão dentro de /wiki/Shattered_Pixel_Dungeon
        for a in response.css('a::attr(href)'):
            href = a.get()
            if href and href.startswith('/wiki/Shattered_Pixel_Dungeon'):
                yield response.follow(a, self.parse)

# Configurando e executando o spider
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
})

process.crawl(ShatteredPixelSpider)
process.start()


2024-09-01 10:44:43 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-09-01 10:44:43 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Windows-11-10.0.22631-SP0
2024-09-01 10:44:43 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-01 10:44:43 [py.warnings] WARNING: c:\Sandbox\Shattered\.venv\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of th